

## Ray Tracing with satGEM grids
- The first task is to reconstruct the ray equations in 4 dimensions (x, y, z, and time) from the original ray tracing  equations where flow and buoyancy only varied with depth.

### Notes from Buhler et al. 1999 paper
- In this paper the x and y planes are aligned and a fully zonal mean flow is assumed. With these assumptions they start with an absolute frequency function:

$$ \Omega(z,k,m) \equiv + \sqrt{\bigg( N^2 \frac{k^2}{k^2 +m^2} + f^2 \bigg)} + U(z)k $$

where rate of change of wavepacket parameters are given by:

$$ \frac{d_gx}{dt} \equiv + \bigg(\frac{\delta\Omega}{\delta k}\bigg)$$


$$ \frac{d_gz}{dt} \equiv + \bigg(\frac{\delta\Omega}{\delta m}\bigg)$$

which are computed as:

$$\frac{d_gx}{dt} \equiv \frac{\omega_0^2 - f^2}{\omega} \frac{m^2}{k(k^2 + m^2)} + U(z) $$

$$\frac{d_gz}{dt} \equiv \frac{\omega_0^2 - f^2}{\omega} \frac{-m}{(k^2 + m^2)} + 0 $$

where zero indicates no vertical flow <span  style="color:red">**Not sure about this**</span>.

and $\omega_0 = \omega - U(z)k$

and:


$$ \frac{d_gk}{dt} \equiv  -\bigg(\frac{\delta\Omega}{\delta x}\bigg) \equiv 0 $$

$$ \frac{d_gm}{dt} \equiv  -\bigg(\frac{\delta\Omega}{\delta z}\bigg) \equiv -k\frac{\delta U}{\delta z}$$




### Revised equations
In order to utilize the satGEM data set, ray equations must be recast with no assumptions about flow in time and space except no important vertical flow **(Seems wrong)** 
My approach to this is to basically add the $l$ term (meridional component) back to the equations instead of assuming its derivatives in time and space are constant/0.

The revised 4D ray equations (group speeds) are:
<hr>
$$\frac{d_gx}{dt} \equiv \frac{\omega_0^2 - f^2}{\omega} \frac{m^2}{k(k^2 + l^2 m^2)} + U(x,y,z) $$

$$\frac{d_gy}{dt} \equiv \frac{\omega_0^2 - f^2}{\omega}
\frac{m^2}{l(k^2 + l^2 m^2)} + V(x,y,z) $$

$$\frac{d_gz}{dt} \equiv \frac{\omega_0^2 - f^2}{\omega} \frac{-m}{(k^2 + m^2 +l^2)} + 0 $$
<hr>

and rate of change of wave parameters:

<span  style="color:blue">**How will $k$ and $l$ vary in space and time now?**</span>.

$$ \frac{d_gk}{dt} \equiv  -\bigg(\frac{\delta\Omega}{\delta x}\bigg) \equiv 0 $$

** Why does vertical wavenumner vary with shear?**

$$ \frac{d_gm}{dt} \equiv  -\bigg(\frac{\delta\Omega}{\delta z}\bigg) \equiv  -k\frac{\delta U(x,y,z,t)}{\delta z}$$


